In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

sys.path.append("/output/models/research/")

import hashlib
import io
import logging
import os
import random
import re

from lxml import etree
import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util


def get_examples(img_path):
    
    label_path =os.path.splitext(img_path)[0]+'.txt'
    if os.path.exists(label_path) is False:
        return False,None
    
    with tf.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()
        print(len(encoded_jpg))
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        print("file format error "+img_path)
        return False,None
    key = hashlib.sha256(encoded_jpg).hexdigest()    
    examples=[]
    for line in open(label_path):  
        data=line.split( )
        xmin = []
        ymin = []
        xmax = []
        ymax = []
        classes = []
        classes_text = []
        difficult_obj=[]
        truncated = []
        poses = []
        
        width=data[0]
        height=data[1]
        file_name=data[2]
        image_format=data[3]
        xmin.append(float(data[4]))
        xmax.append(float(data[5]))
        ymin.append(float(data[6]))
        ymax.append(float(data[7]))
        classes.append(int(data[8]))
        classes_text.append(data[9].encode('utf8'))

        
        print(file_name)
        
        example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset_util.int64_feature(int(height)),
          'image/width': dataset_util.int64_feature(int(width)),
          'image/filename': dataset_util.bytes_feature(file_name.encode('utf8')),
          'image/source_id': dataset_util.bytes_feature(file_name.encode('utf8')),
          'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
          'image/encoded': dataset_util.bytes_feature(encoded_jpg),
          'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
          'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
          'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
          'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
          'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
          'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
          'image/object/class/label': dataset_util.int64_list_feature(classes),
        }))
        examples.append(example)

    return True,examples    

def create_tf_record(examples_list,output_filename):
    writer = tf.python_io.TFRecordWriter(output_filename)
    for i in examples_list:
        ret,examples=get_examples(i)
        if ret:
            for j in examples:
                writer.write(j.SerializeToString())
    writer.close()   


def main(image_dir='./image/',tfrecord_dir='./',example_percent=0.7):
    images=[]
    for (root,dirs,files) in os.walk(image_dir) :
        for item in files:
            if item.endswith('jpg'):
                images.append(os.path.join(root, item))
    
    random.seed(42)
    random.shuffle(images)
    num_examples = len(images)
    num_train = int(example_percent* num_examples)
    train_examples = images[:num_train]
    val_examples = images[num_train:]
    logging.info('%d training and %d validation examples.',len(train_examples), len(val_examples))
    
    train_output_path = os.path.join(tfrecord_dir, 'train.record')
    val_output_path = os.path.join(tfrecord_dir, 'val.record')
    
    create_tf_record(train_examples,train_output_path)
    create_tf_record(val_examples,val_output_path)
    logging.info('%d training and %d validation examples.')
    
    
main()

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


10366
dog29.jpg
9559
dog279.jpg
9441
dog31.jpg
15784
dog140.jpg
10609
dog187.jpg
11655
dog158.jpg
11878
dog226.jpg
12247
dog301.jpg
17173
dog234.jpg
8654
dog19.jpg
8627
dog311.jpg
12886
dog117.jpg
10178
dog283.jpg
7452
dog262.jpg
9511
dog217.jpg
12683
dog0.jpg
13812
dog250.jpg
7470
dog143.jpg
7874
dog195.jpg
12218
dog129.jpg
7624
dog215.jpg
11624
dog10.jpg
10473
dog278.jpg
12270
dog17.jpg
21031
dog200.jpg
13926
dog14.jpg
8853
dog276.jpg
12544
dog294.jpg
12218
dog180.jpg
12670
dog249.jpg
8436
dog184.jpg
12874
dog100.jpg
19679
dog222.jpg
9883
dog167.jpg
11377
dog130.jpg
8571
dog221.jpg
11372
dog112.jpg
12391
dog296.jpg
6450
dog182.jpg
19084
dog289.jpg
8908
dog277.jpg
11797
dog22.jpg
11605
dog207.jpg
9945
dog236.jpg
9758
dog28.jpg
9545
dog287.jpg
9577
dog224.jpg
13825
dog115.jpg
18572
dog203.jpg
11380
dog186.jpg
8148
dog169.jpg
9547
dog124.jpg
11624
dog197.jpg
13940
dog20.jpg
11524
dog179.jpg
9683
dog229.jpg
11132
dog210.jpg
13296
dog286.jpg
19650
dog136.jpg
8487
dog211.jpg
12854
dog264.j